In [ ]:
!wget https://raw.githubusercontent.com/alexmascension/revisit_reynolds_fb/master/requirements.txt
!pip install -r requirements.txt

In [ ]:
import scanpy as sc
import scanpy.external as sce

import pandas as pd
import numpy as np

import os

import triku as tk

import matplotlib.pyplot as plt
import matplotlib as mpl

# TODO
* Adapt adata loading from URL
* Intro + conclusions  

In [ ]:
# To print versions of imports 

import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    try:
        name, version = module.split('==')
        if name in changed_imported_modules:
            print(name + '\t' + version)
    except:
        pass

In [ ]:
seed = 0

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 150

In [ ]:
def assign_cats(adata, dict_cats, column_groupby='leiden', per=98):
    """
    This functions uses a set of genes assigned to different categories so that leiden clusters can be assigned to one of these categories.
    For example, to categorize fibroblasts from pericytes, endothelial cells, or cells with high mitochondrial content.
    It could be done with each cell individually, but it is better to use clusters to discern the different categories because
    the method, although efficient, can sometimes be noisy due to the noisiness of the sc datasets.
    """
    for cat in list(dict_cats.keys()):
        mat_cat = adata[:, dict_cat[cat]].X.copy().todense()

        for c in range(mat_cat.shape[1]):
            mat_cat[:, c] /= np.percentile(mat_cat[:, c][mat_cat[:, c] > 0], per)

        sum_mat_cat = np.asarray(mat_cat.sum(1)).ravel()
        sum_mat_cat /= np.percentile(sum_mat_cat[sum_mat_cat > 0], per, axis=0)

        adata.obs[cat] = sum_mat_cat
    
    score_per_cluster = adata.obs[[column_groupby] + list(dict_cats.keys())].groupby(column_groupby).mean()
    max_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, score_per_cluster.idxmax(axis=1).values))
    adata.obs['assigned_cats'] = [max_cat_dict[i] for i in adata.obs[column_groupby]]

## Reynolds et al. 2020

In [ ]:
os.getcwd()

In [ ]:
reynolds_dir = 'reynolds_2020'
os.makedirs(reynolds_dir, exist_ok=True)

In [ ]:
# Making and saving the fb healthy dataset to zenodo
adata_reynolds = sc.read('submission_210120.h5ad', backup_url='https://zenodo.org/record/4536165/files/submission_210120.h5ad')
adata_reynolds_fb = adata_reynolds[(adata_reynolds.obs['full_clustering'].isin(['F1', 'F2', 'F3'])) & 
                                   (adata_reynolds.obs['Status'] == 'Healthy')]
sc.pp.filter_genes(adata_reynolds_fb, min_counts=100)
del adata_reynolds_fb.var

for obs in ['mad_prd', 'Status', 'Site', 'Tissue', 'Enrichment', 'Location', 'Sex', 'Age', 'stage']:
    del adata_reynolds_fb.obs[obs]
    
adata_reynolds_fb.write_h5ad(reynolds_dir + '/reynolds_2020_fb_healthy.h5ad')

### Direct h5ad download

In [ ]:
#!wget https://zenodo.org/record/4536165/files/submission_210120.h5ad -o {reynolds_dir}/reynolds_2020.h5ad
# adata_reynolds = sc.read_h5ad(reynolds_dir+'/reynolds_2020.h5ad')

In [ ]:
adata_reynolds_fb_healthy = sc.read(reynolds_dir + '/reynolds_2020_fb_healthy.h5ad', backup_url='https://zenodo.org/record/4605340/files/reynolds_2020_fb_healthy.h5ad?download=1')

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_healthy, min_counts=50)

In [ ]:
sc.pp.log1p(adata_reynolds_fb_healthy)
sc.pp.normalize_total(adata_reynolds_fb_healthy)

In [ ]:
df_batches = pd.DataFrame(np.unique(adata_reynolds_fb_healthy.obs['sample_id'], return_counts=True)).transpose()

In [ ]:
df_batches.sort_values(by=1, ascending=False)

In [ ]:
selected_samples = df_batches[df_batches[1] > 50][0].values

In [ ]:
adata_reynolds_fb_healthy = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(selected_samples)]  #selected_samples)]

In [ ]:
adata_reynolds_fb_healthy

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_healthy, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_healthy, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_reynolds_fb_healthy, metric='angular', batch_key='sample_id')
tk.tl.triku(adata_reynolds_fb_healthy, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(adata_reynolds_fb_healthy, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_healthy, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

## 4820STDY7388991

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7388991 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7388991'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7388991

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom = sc.read('reynolds_2020/reynolds_2020_0_4820STDY7388991_s1_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_0_4820STDY7388991_s1_dermis_fibroblasts.loom')
adata_reynolds_fb_4820STDY7388991_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7388991_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_4820STDY7388991_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7388991.obs_names & adata_reynolds_fb_4820STDY7388991_loom.obs_names] = adata_reynolds_fb_4820STDY7388991[adata_reynolds_fb_4820STDY7388991.obs_names & adata_reynolds_fb_4820STDY7388991_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7388991_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7388991_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7388991_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7388991_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7388991_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7388991_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991_loom, resolution=3.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['LUM', 'PDGFRA', 'VIM', 'DCN', 'COL1A1', 'SFRP2', 'APOE', 'POSTN'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom)

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TPSB2', 'TPSAB1', 'HLA-DRA', 'FCER1G', 'CD74'], 'melanocyte': ['PMEL', 'MLANA'], 
            'mt': ['MTND2P28', 'MTND4P12', 'MTCO1P40', 'ADAM33', 'RN7SL2', 'MTRNR2L6']}

assign_cats(adata_reynolds_fb_4820STDY7388991_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom_fb = adata_reynolds_fb_4820STDY7388991_loom[adata_reynolds_fb_4820STDY7388991_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991_loom_fb, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991_loom_fb, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991_loom_fb, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991_loom_fb, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, min_dist=0.2, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991_loom_fb, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1', 'POSTN'], cmap=magma, use_raw=False, legend_loc='on data')

## 4820STDY7388999

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7388999 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7388999'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7388999

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom = sc.read('reynolds_2020/reynolds_2020_8_4820STDY7388999_s2_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_8_4820STDY7388999_s2_dermis_fibroblasts.loom')

adata_reynolds_fb_4820STDY7388999_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7388999_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('Other', index=adata_reynolds_fb_4820STDY7388999_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7388999.obs_names & adata_reynolds_fb_4820STDY7388999_loom.obs_names] = adata_reynolds_fb_4820STDY7388999[adata_reynolds_fb_4820STDY7388999.obs_names & adata_reynolds_fb_4820STDY7388999_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7388999_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7388999_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7388999_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7388999_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7388999_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7388999_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999_loom, resolution=0.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 
                                                          'DMKN', 'KRT1'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TPSB2', 'TRAC', 'TPSAB1', 'HLA-DRA', 'FCER1G', 'CD74'], 'endo': ['CLDN5', 'PECAM1'], 'kerato': ['DMKN', 'KRT1'],
            'mt': ['MTND2P28', 'MTND4P12', 'MTCO1P40', 'ADAM33', 'RN7SL2', 'MTRNR2L6']}

assign_cats(adata_reynolds_fb_4820STDY7388999_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom)

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom = adata_reynolds_fb_4820STDY7388999_loom[adata_reynolds_fb_4820STDY7388999_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1', 'EDNRB', 'IGFBP7'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom, groupby='leiden')

In [ ]:
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom)

## 4820STDY7389007

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7389007 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7389007'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7389007

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom = sc.read('reynolds_2020/reynolds_2020_16_4820STDY7389007_s3_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_16_4820STDY7389007_s3_dermis_fibroblasts.loom')

adata_reynolds_fb_4820STDY7389007_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7389007_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_4820STDY7389007_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7389007.obs_names & adata_reynolds_fb_4820STDY7389007_loom.obs_names] = adata_reynolds_fb_4820STDY7389007[adata_reynolds_fb_4820STDY7389007.obs_names & adata_reynolds_fb_4820STDY7389007_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7389007_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7389007_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7389007_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7389007_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7389007_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7389007_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007_loom, resolution=0.8, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TPSB2', 'TRAC', 'TPSAB1', 'HLA-DRA', 'FCER1G'], 'kerato': ['DMKN', 'KRT1'],
            'mt': ['MTND2P28', 'MTND4P12', 'MTCO1P40', 'ADAM33', 'RN7SL2', 'MTRNR2L6']}

assign_cats(adata_reynolds_fb_4820STDY7389007_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7389007_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7389007_loom)

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom = adata_reynolds_fb_4820STDY7389007_loom[adata_reynolds_fb_4820STDY7389007_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

## SKN8104899

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_SKN8104899 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['SKN8104899'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_SKN8104899

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_SKN8104899, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_SKN8104899_loom = sc.read('reynolds_2020/reynolds_2020_84_SKN8104899_S4_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_84_SKN8104899_S4_dermis_fibroblasts.loom')

adata_reynolds_fb_SKN8104899_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_SKN8104899_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_SKN8104899_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_SKN8104899_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_SKN8104899.obs_names & adata_reynolds_fb_SKN8104899_loom.obs_names] = adata_reynolds_fb_SKN8104899[adata_reynolds_fb_SKN8104899.obs_names & adata_reynolds_fb_SKN8104899_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_SKN8104899_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_SKN8104899_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_SKN8104899_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_SKN8104899_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_SKN8104899_loom)
sc.pp.log1p(adata_reynolds_fb_SKN8104899_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8104899_loom, resolution=0.8, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 'POSTN'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TRAC', 'HLA-DRA', 'FCER1G'], 'kerato': ['DMKN', 'KRT1'],
            'mt': ['MTND4P12', 'ADAM33', 'RN7SL2', ]}

assign_cats(adata_reynolds_fb_SKN8104899_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom)

In [ ]:
adata_reynolds_fb_SKN8104899_loom = adata_reynolds_fb_SKN8104899_loom[adata_reynolds_fb_SKN8104899_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8104899_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

## SKN8105197

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_SKN8105197 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['SKN8105197'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_SKN8105197

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_SKN8105197, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_SKN8105197_loom = sc.read('reynolds_2020/reynolds_2020_92_SKN8105197_S5_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_92_SKN8105197_S5_dermis_fibroblasts.loom')
adata_reynolds_fb_SKN8105197_loom.var_names_make_unique()

In [ ]:
adata_reynolds_fb_SKN8105197_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_SKN8105197_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_SKN8105197_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_SKN8105197.obs_names & adata_reynolds_fb_SKN8105197_loom.obs_names] = adata_reynolds_fb_SKN8105197[adata_reynolds_fb_SKN8105197.obs_names & adata_reynolds_fb_SKN8105197_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_SKN8105197_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_SKN8105197_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_SKN8105197_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_SKN8105197_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197_loom, min_counts=30)
sc.pp.normalize_per_cell(adata_reynolds_fb_SKN8105197_loom)
sc.pp.log1p(adata_reynolds_fb_SKN8105197_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8105197_loom, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'SFRP2', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['HLA-DRA', 'FCER1G'], 'melano':['PMEL', 'MLANA']}

assign_cats(adata_reynolds_fb_SKN8105197_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8105197_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_SKN8105197_loom)

In [ ]:
adata_reynolds_fb_SKN8105197_loom = adata_reynolds_fb_SKN8105197_loom[adata_reynolds_fb_SKN8105197_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197_loom, metric='cosine')

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8105197_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'SFRP2', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

### SKN8105197 exclusion
We do not see SFRP2 expression, which has been a through fb marker, together with APOE. We will not include this sample in the analysis.